In [1]:
import cv2
import numpy as np
from pygame import mixer
from tensorflow.keras.models import load_model
import cv2.data

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html



In [2]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [4]:
model = load_model(r'D:\Yogesh\Projects\Project 10 EyeGuard\model_checkpoint\model.h5')

In [5]:
mixer.init()

sound = mixer.Sound(r'D:\Yogesh\Projects\Project 10 EyeGuard\s.mp3')

In [6]:
cap = cv2.VideoCapture(0)
score = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 3)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        eyes = eye_cascade.detectMultiScale(roi_gray)

        for (ex, ey, ew, eh) in eyes:
            eye = roi_color[ey:ey+eh, ex:ex+ew]

            eye = cv2.resize(eye, (80, 80)) / 255.0
            eye = np.expand_dims(eye, axis=0)

            prediction = model.predict(eye)

            if prediction[0][0] > 0.50:
                cv2.putText(frame, 'closed', (10, frame.shape[0]-20), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 1)
                score += 1
                if score > 5:
                    try:
                        sound.play()
                    except:
                        pass
            else:
                cv2.putText(frame, 'open', (10, frame.shape[0]-20), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 255, 255), 1)
                if score > 0:
                    score -= 1

    cv2.imshow('frame', frame)

    if cv2.waitKey(33) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 41ms/step
